In [1]:
!pip install kaggle

Importing the dependencies

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection -Kaggle API

In [4]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [5]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [6]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [7]:
!ls

kaggle.json  sample_data


In [17]:
# Refresh the file system view within the Jupyter Notebook
!ls

# unzip the dataset file
with ZipFile("/content/IMDB Dataset.csv.zip", "r") as zip_ref:
  zip_ref.extractall()

'IMDB Dataset.csv.zip'	 kaggle.json   sample_data


In [18]:

!ls

'IMDB Dataset.csv'  'IMDB Dataset.csv.zip'   kaggle.json   sample_data


In [20]:

data = pd.read_csv("/content/IMDB Dataset.csv.zip")

In [21]:

data.shape

(50000, 2)

In [22]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [23]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [24]:

data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [25]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [26]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [27]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [28]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [29]:

print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [31]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [32]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [33]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [34]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [35]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM - Long Short-Term Memory

In [36]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:

# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Training the Model

In [40]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 392s 784ms/step - loss: 0.3476 - accuracy: 0.8552 - val_loss: 0.3349 - val_accuracy: 0.8624
Epoch 2/5
500/500 [==============================] - 397s 794ms/step - loss: 0.2604 - accuracy: 0.8961 - val_loss: 0.2927 - val_accuracy: 0.8777
Epoch 3/5
500/500 [==============================] - 391s 783ms/step - loss: 0.2130 - accuracy: 0.9174 - val_loss: 0.3026 - val_accuracy: 0.8720
Epoch 4/5
500/500 [==============================] - 395s 791ms/step - loss: 0.1851 - accuracy: 0.9296 - val_loss: 0.3268 - val_accuracy: 0.8779
Epoch 5/5
500/500 [==============================] - 406s 811ms/step - loss: 0.1630 - accuracy: 0.9380 - val_loss: 0.3295 - val_accuracy: 0.8755


Model Evaluation

In [41]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 [==============================] - 29s 92ms/step - loss: 0.3158 - accuracy: 0.8805
Test Loss: 0.3157525360584259
Test Accuracy: 0.8805000185966492


Building a Predictive System

In [42]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [43]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 336ms/step
The sentiment of the review is: positive


In [44]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 53ms/step
The sentiment of the review is: negative


In [ ]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")